Algo used: 1 for epsilon greedy, 2 for softmax, 3 for UCB1 and 4 Median Elimination

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import softmax

np.random.seed(0)
##### Generating means for all arms from Guassian distribution(0,1)
def generate_mean_for_arms(mean, variance, number_of_arms):
    return np.random.normal(mean,np.sqrt(variance),(number_of_arms))

#########Implementation of epsilon greedy algo
def generate_epsilon_greedy_run(number_of_steps,number_of_arms, epsilon, epsilon_decrease_allow ):
    #Actual mean of arms 
    mean_for_arms = generate_mean_for_arms(0, 1, number_of_arms)
    #Actual optimal action
    optimal_action = np.argmax(mean_for_arms)
    #Running expectead mean of arms
    expected_mean_for_arms = np.zeros((number_of_arms))*1.0
    #Counting the number of times an arm is selected
    number_of_times_arm_selected = np.zeros((number_of_arms))
    #Keeping the record wether Actual optimal action is chosen at time step t or not
    optimal_action_chosen_at_step = np.zeros((number_of_steps))
    #Keeping the reward at each time step
    run = []
    for i in range(number_of_steps):
        if epsilon_decrease_allow == 1:
            if i%10 == 0:
                epsilon = epsilon - .99900e-4
        r = np.random.uniform(0,1)
        if r<epsilon:
            action_chosen = np.random.randint(0,number_of_arms)
            if action_chosen == optimal_action:
                optimal_action_chosen_at_step[i] = optimal_action_chosen_at_step[i]+1
            reward = np.random.normal(mean_for_arms[action_chosen],1)
            run.append(reward)
            
            number_of_times_arm_selected[action_chosen]= number_of_times_arm_selected[action_chosen] +1
            expected_mean_for_arms[action_chosen] = expected_mean_for_arms[action_chosen] + (reward - expected_mean_for_arms[action_chosen])/number_of_times_arm_selected[action_chosen]
            # run.append(expected_mean_for_arms[action_chosen])
        else:
            action_chosen = np.argmax(expected_mean_for_arms)
            if action_chosen == optimal_action:
                optimal_action_chosen_at_step[i] = optimal_action_chosen_at_step[i]+1
            
            reward = np.random.normal(mean_for_arms[action_chosen],1)
            run.append(reward)
            
            number_of_times_arm_selected[action_chosen]= number_of_times_arm_selected[action_chosen] +1
            expected_mean_for_arms[action_chosen] = expected_mean_for_arms[action_chosen] + (reward - expected_mean_for_arms[action_chosen])/number_of_times_arm_selected[action_chosen]
            # run.append(expected_mean_for_arms[action_chosen])
        
    return run, np.argmax(mean_for_arms), np.argmax(expected_mean_for_arms), number_of_times_arm_selected, optimal_action_chosen_at_step




def generate_softamx_run(number_of_steps,number_of_arms, temperature):
    
    mean_for_arms = generate_mean_for_arms(0, 1, number_of_arms)
    optimal_action = np.argmax(mean_for_arms)
    expected_mean_for_arms = np.zeros((number_of_arms))*1.0
    number_of_times_arm_selected = np.zeros((number_of_arms))
    optimal_action_chosen_at_step = np.zeros((number_of_steps))
    run = []
    for i in range(number_of_steps):
        
        softprob = softmax(expected_mean_for_arms/temperature)
        
        action_chosen = np.random.choice(np.arange(number_of_arms),p = softprob)
        # print("action_chosen", action_chosen)
        if action_chosen == optimal_action:
            optimal_action_chosen_at_step[i] = optimal_action_chosen_at_step[i]+1
        reward = np.random.normal(mean_for_arms[action_chosen],1)
        run.append(reward)
        
        number_of_times_arm_selected[action_chosen]= number_of_times_arm_selected[action_chosen] +1
        expected_mean_for_arms[action_chosen] = expected_mean_for_arms[action_chosen] + (reward - expected_mean_for_arms[action_chosen])/number_of_times_arm_selected[action_chosen]
            # run.append(expected_mean_for_arms[action_chosen])
      
        
    return run, np.argmax(mean_for_arms), np.argmax(expected_mean_for_arms), number_of_times_arm_selected, optimal_action_chosen_at_step


def generate_UCB_run(number_of_steps, number_of_arms, c):
    
    mean_for_arms = generate_mean_for_arms(0, 1, number_of_arms)
    optimal_action = np.argmax(mean_for_arms)
    expected_mean_for_arms = np.zeros((number_of_arms))*1.0
    number_of_times_arm_selected = np.ones((number_of_arms))
    optimal_action_chosen_at_step = np.zeros((number_of_steps))
    run = []
    for i in range(number_of_steps):
    
        # number_of_arms = 10
        a= np.repeat(np.log(i+1),number_of_arms)
        b = number_of_times_arm_selected
        d = c*np.sqrt(a/b) + expected_mean_for_arms
        action_chosen = np.argmax(d)
        if action_chosen == optimal_action:
            optimal_action_chosen_at_step[i] = optimal_action_chosen_at_step[i]+1
        reward = np.random.normal(mean_for_arms[action_chosen],1)
        run.append(reward)
        
        number_of_times_arm_selected[action_chosen]= number_of_times_arm_selected[action_chosen] +1
        expected_mean_for_arms[action_chosen] = expected_mean_for_arms[action_chosen] + (reward - expected_mean_for_arms[action_chosen])/number_of_times_arm_selected[action_chosen]
            
        
    return run, np.argmax(mean_for_arms), np.argmax(expected_mean_for_arms), number_of_times_arm_selected, optimal_action_chosen_at_step


     

from datetime import datetime



def generate_median_elimination(number_of_arms,epsilon, delta):
    mean_for_arms = generate_mean_for_arms(0, 1, number_of_arms)
    optimal_action = np.argmax(mean_for_arms)
    expected_mean_for_arms = np.zeros((number_of_arms))*1.0
    epsilon_1 = epsilon/4
    
    times_to_pull_list_last = 0
    delta_1 = delta/2
    armlist = np.arange(number_of_arms)
    total_count_samples = 0
    is_optimal_action_retained = 0
    time_for_sort = 0
    optimal_action_chosen_at_step = []
    run =[]
    w = datetime.now()
    while(len(armlist)>1):
        samply = []
        steps = []
        times_to_pull = int(1/((epsilon_1/2)**2)*np.log10(3/delta_1))
        total_count_samples = total_count_samples + times_to_pull*len(armlist)
        for a in armlist:
            samples = np.random.normal(mean_for_arms[a],1,(times_to_pull))
            
            samply = np.concatenate((samply,samples))
            if int(a) == optimal_action:
                
                steps = np.concatenate((steps, np.ones(len(samples))))
            else:
                steps = np.concatenate((steps, np.zeros(len(samples))))
            sums = np.sum(samples)
            
            expected_mean_for_arms[a]  = (expected_mean_for_arms[a]*times_to_pull_list_last +  sums)/(times_to_pull_list_last +times_to_pull )
        np.random.shuffle(samply)
        np.random.shuffle(steps)
        optimal_action_chosen_at_step = np.concatenate((optimal_action_chosen_at_step, steps))
        run = np.concatenate((run,samply))
        # print("run",np.array(run).shape)
        times_to_pull_list_last = times_to_pull_list_last + times_to_pull
        arr = np.array(expected_mean_for_arms)
        
        c = datetime.now()
        q = arr.argsort()[-int(np.ceil(len(armlist)/2)):][::-1]
        d = datetime.now()
        time_for_sort = time_for_sort + (d-c).total_seconds()
        armlist = q
        # print("armlist",armlist)
        epsilon_1 = epsilon_1*(3/4)
        delta_1 = delta_1/2
        # print("loop end")
    if optimal_action == int(armlist[0]):
       is_optimal_action_retained = 1 
    b = datetime.now()
    # print("time for algo, time for sort, time for other then algo",(b-w).total_seconds(),time_for_sort, (b-w).total_seconds()-time_for_sort )
    return optimal_action, armlist[0], total_count_samples, is_optimal_action_retained, run, optimal_action_chosen_at_step



################Generating Average run over given number of Bandit problems
def generate_average_run(number_of_bandits, number_of_steps,number_of_arms,algo_used,epsilon = 0,alpha = .1, temperature = 100, c = 1, epsilon_decrease_allow = 0,delta = .1):
    
        all_run = []
        run = []
        all_percent_optimal_action_chosen = []
        optimal_action_chosen_at_step = []
        optimal_action_retained_count = 0
        total_count_samples = 0
        for i in range(number_of_bandits):
        
            if algo_used == 1:
                run, actual_best_arm,expected_best_arm, number_of_times_arm_selected, optimal_action_chosen_at_step = generate_epsilon_greedy_run(number_of_steps, number_of_arms,epsilon, epsilon_decrease_allow)
            elif algo_used == 2:
                run, actual_best_arm,expected_best_arm, number_of_times_arm_selected, optimal_action_chosen_at_step = generate_softamx_run(number_of_steps,number_of_arms,temperature)
            elif algo_used == 3:
                run, actual_best_arm,expected_best_arm, number_of_times_arm_selected, optimal_action_chosen_at_step  = generate_UCB_run(number_of_steps, number_of_arms, c)
            elif algo_used == 4:
                optimal_action, arm, total_count_samples, is_optimal_action_retained, run, optimal_action_chosen_at_step = generate_median_elimination(number_of_arms,epsilon, delta)
                optimal_action_retained_count = optimal_action_retained_count + is_optimal_action_retained
                # print("jhvjh", np.array(run).shape)

            # print("yo",actual_best_arm,expected_best_arm, number_of_times_arm_selected)
            all_run.append(run)
            all_percent_optimal_action_chosen.append(optimal_action_chosen_at_step)
        # print("ewd",np.array(all_percent_optimal_action_chosen).shape)
        return np.mean(all_run,0), (np.sum(all_percent_optimal_action_chosen,0)/number_of_bandits)*100, total_count_samples, (optimal_action_retained_count/number_of_bandits)*100



In [0]:
################Plot for question 1

def plot1(number_of_bandits,number_of_steps,number_of_arms):
    x = np.arange(number_of_steps)
    y1, z1,_, _ = generate_average_run(number_of_bandits,number_of_steps, number_of_arms,1,0)
    y2, z2, _,_ = generate_average_run(number_of_bandits,number_of_steps,number_of_arms,1,.1)
    y3, z3,_,_ = generate_average_run(number_of_bandits,number_of_steps,number_of_arms,1,.01)
    y4, z4,_,_ = generate_average_run(number_of_bandits,number_of_steps,number_of_arms,1,.1,epsilon_decrease_allow=1)
    
    plt.figure(figsize=(20,10))
    plt.title("Average performance of epsilon-greedy action-value methods on the 1000-armed testbed", fontsize=20)
    plt.ylabel("Average reward", fontsize=20)
    plt.xlabel("Steps", fontsize=20)
    l1,=  plt.plot(x,y1, label = "$\epsilon$ 0")
    l2, =  plt.plot(x,y2, label = "$\epsilon$ 0.1")
    l3, =  plt.plot(x,y3, label = "$\epsilon$ 0.01")
    l4, =  plt.plot(x,y4, label = "$\epsilon$ 0.1 -.0001 ")
    
    plt.legend(handles=[l1,l2,l3,l4], prop={'size': 18})
    plt.show()
    plt.close()
    plt.figure(figsize=(20,10))
    plt.ylabel("% Optimal action",fontsize=20)
    plt.xlabel("Steps", fontsize=20)
    l1, =  plt.plot(x,z1, label = "$\epsilon$ 0")
    l2, =  plt.plot(x,z2, label = "$\epsilon$ 0.1")
    l3, =  plt.plot(x,z3, label = "$\epsilon$ 0.01")
    l4, =  plt.plot(x,z4, label = "$\epsilon$ 0.1 - .0001")
   
    plt.legend(handles=[l1,l2,l3, l4], prop={'size': 18})
    plt.show()
    plt.close()

# plot for question2
def plot2(number_of_bandits, number_of_steps,number_of_arms):
    x = np.arange(number_of_steps)
    y1, z1,_,_ = generate_average_run(number_of_bandits, number_of_steps,number_of_arms,2,temperature = .005)
    y2, z2,_,_ = generate_average_run(number_of_bandits, number_of_steps,number_of_arms,2,temperature = .01)
    y3, z3,_,_ = generate_average_run(number_of_bandits, number_of_steps,number_of_arms, 2,temperature = .05)
    y4, z4,_,_ = generate_average_run(number_of_bandits, number_of_steps,number_of_arms,2,temperature = .1)
    y5, z5,_,_ = generate_average_run(number_of_bandits, number_of_steps,number_of_arms, 2,temperature = .3)
    y6, z6,_,_ = generate_average_run(number_of_bandits, number_of_steps,number_of_arms, 2,temperature = .5)
    y7, z7,_,_ = generate_average_run(number_of_bandits, number_of_steps,number_of_arms, 2,temperature = 100)

    plt.figure(figsize=(20,10))
    plt.title("Average performance of softmax action-value methods on the 10-armed testbed",  fontsize=20)
    plt.ylabel("Average reward",  fontsize=20)
    plt.xlabel("Steps" , fontsize=20)
    l1,=  plt.plot(x,y1, label = "temperature = .005")
    l2, =  plt.plot(x,y2, label = "temperature = .01")
    l3, =  plt.plot(x,y3, label = "temperature = .05")
    l4, =  plt.plot(x,y4, label = " temperature = .1")
    l5, =  plt.plot(x,y5, label = "temperature = .3")
    l6, =  plt.plot(x,y6, label = "temperature = .5")
    l7, =  plt.plot(x,y7, label = "temperature = 100")
    
    plt.legend(handles=[l1,l2,l3, l4,l5,l6, l7], prop={'size': 18})
    plt.show()
    plt.close()
    plt.figure(figsize=(20,10))
    plt.ylabel("% Optimal action",  fontsize=20)
    plt.xlabel("Steps",  fontsize=20)
    l1,=  plt.plot(x,z1, label = "temperature = .005")
    l2, =  plt.plot(x,z2, label = "temperature = .01")
    l3, =  plt.plot(x,z3, label = "temperature = .05")
    l4, =  plt.plot(x,z4, label = " temperature = .1")
    l5, =  plt.plot(x,z5, label = "temperature = .3")
    l6, =  plt.plot(x,z6, label = "temperature = .5")
    l7, =  plt.plot(x,z7, label = "temperature = 100")
   
    plt.legend(handles=[l1,l2,l3, l4, l5,l6, l7], prop={'size': 18})
    plt.show()
    plt.close()
    
# plot for question 3
def plot3(number_of_bandits,number_of_steps,number_of_arms):
    x = np.arange(number_of_steps)
    y1, z1,_,_ = generate_average_run(number_of_bandits, number_of_steps, number_of_arms,1,.1)
    y2, z2,_,_ = generate_average_run(number_of_bandits, number_of_steps,number_of_arms,2,temperature = .3)
    y3, z3,_,_ = generate_average_run(number_of_bandits, number_of_steps,number_of_arms,3,c = 2)
    # _,_,y4, z4 = generate_average_run(number_of_bandits, number_of_steps,number_of_arms,4,epsilon = .99, delta = .99)

    plt.figure(figsize=(20,10))
    plt.title("Comparison of performance of all four action-value methods on the 10-armed testbed", fontsize=20)
    plt.ylabel("Average reward", fontsize=20)
    plt.xlabel("Steps", fontsize=20)
    l1,=  plt.plot(x,y1, label = " Epsilon Greedy $\epsilon$ 0.1")
    l2, =  plt.plot(x,y2, label = " Softmax temperature .3")
    l3, =  plt.plot(x,y3, label = " UCB  c = 2")
    # l4, =  plt.plot(x,y4, label = " MEDIAN \epsilon = .99 \delta = .99")
    
    plt.legend(handles=[l1,l2,l3], prop={'size': 18})
    plt.show()
    plt.close()
    plt.figure(figsize=(20,10))
    plt.ylabel("% Optimal action", fontsize=20)
    plt.xlabel("Steps", fontsize=20)
    l1,=  plt.plot(x,z1, label = " Epsilon Greedy $\epsilon$ 0.1")
    l2, =  plt.plot(x,z2, label = " Softmax temperature = .3")
    l3, =  plt.plot(x,z3, label = " UCB  c = 2")
    # l4, =  plt.plot(x,z4, label = " MEDIAN \epsilon = .99 \delta = .99")
   
    plt.legend(handles=[l1,l2,l3], prop={'size': 18})
    plt.show()
    plt.close()

####plot for question 4

def plot4(number_of_bandits, number_of_steps,number_of_arms):
    x = [.05, .08, .1, .3, .5]
    # x = [.95, .94, .93,.92, .91]
    y =[]
    z = []

    for i in range(5):
        _,_,y1, z1 = generate_average_run(number_of_bandits, number_of_steps,number_of_arms,4,epsilon = x[i], delta = .05)
        print(y1,z1)
        y.append(y1)
        z.append(z1)

    
  

    plt.figure(figsize=(10,5))
    plt.title("Average performance of Median action-value methods on the 10-armed testbed for different values of epsilon for a given delta = .05",  fontsize=20)
    plt.ylabel("Total number of samples",  fontsize=20)
    plt.xlabel("Epsilon" , fontsize=20)
    plt.plot(x,y)
    plt.show()
    plt.close()

    plt.figure(figsize=(10,5))
    plt.ylabel("% Optimal action retained",  fontsize=20)
    plt.xlabel("Epsilon",  fontsize=20)
    plt.plot(x,z)
    plt.show()
    plt.close()
    print(x,y,z)
 
def plot5(number_of_bandits, number_of_steps,number_of_arms):
    a1,b1,y1, z1 = generate_average_run(number_of_bandits, number_of_steps,number_of_arms,4,epsilon = .7, delta = .05)
  
   
    
    # print(y1)
    # print("shape",np.array(a1).shape,np.array(b1).shape)
    qui = np.min([y1])
    x = np.arange(qui)
    
    plt.figure(figsize=(10,5))
    plt.title("Average performance of Median Elimination  action-value methods on the 10-armed testbed", fontsize=20)
    plt.ylabel("Average reward", fontsize=20)
    plt.xlabel("Steps", fontsize=20)
    l1,=  plt.plot(x,a1[:qui], label = " epsilon = .9, delta = .1")
    
    
    

    plt.legend(handles=[l1], prop={'size': 18})
    plt.show()
    plt.close()
    plt.figure(figsize=(10,5))
    plt.ylabel("% Optimal action", fontsize=20)
    plt.xlabel("Steps", fontsize=20)
    l1,=  plt.plot(x,b1[:qui], label = " epsilon = .9, delta = .1")
    
    
    

    plt.legend(handles=[l1], prop={'size': 18})
    plt.show()
    plt.close()


    
   

    





In [0]:
plot5(2000,1000,10)

In [0]:
plot1(2000,10000, 10)

In [0]:
plot2(2000, 1000, 10)

In [0]:
plot3(2000, 3226,10)